In [16]:
!pip install selenium

In [6]:
import time
import pandas as pd
import os 
import pymysql
from selenium import webdriver 

In [2]:
!pip install webdriver_manager

In [3]:
#크롬드라이버링크삽입
#driver = webdriver.Chrome('chromedriver.exe')
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager



In [4]:


driver = webdriver.Chrome(ChromeDriverManager().install())


driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/')

 # id & pw 입력 
driver.find_element_by_name('id').send_keys('')  #네이버 아이디 
driver.find_element_by_name('pw').send_keys('')  #네이버 패스워드 
 # click login 
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()


 #맘카페 링크 입력 ( 네이버 카페 맘카페 상위 5개 크롤링)                                    
driver.get('https://cafe.naver.com/moms1004')

driver.find_element_by_xpath('//*[@id="SEOneBannerLayerCloseBtn"]').click()
# base_url = cafe main page url



[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/84.0.4147.30/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/kyung/.wdm/drivers/chromedriver/mac64/84.0.4147.30]


In [7]:
driver = webdriver.Chrome(ChromeDriverManager().install())


driver.get('https://nid.naver.com/nidlogin.login?svctype=262144&url=http://m.naver.com/')

 # id & pw 입력 
driver.find_element_by_name('id').send_keys('')  #네이버 아이디 
driver.find_element_by_name('pw').send_keys('')  #네이버 패스워드 
 # click login 
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click()


 #맘카페 링크 입력 ( 네이버 카페 맘카페 상위 5개 크롤링)                                    
driver.get('https://cafe.naver.com/moms1004')

driver.find_element_by_xpath('//*[@id="SEOneBannerLayerCloseBtn"]').click()
# base_url = cafe main page url


base_url ='https://cafe.naver.com/moms1004'
cnt = 0 # number of collected data
page = 0 # position of current page # db connect and select 
conn = pymysql.connect(host='127.0.0.1', user = 'root', password='kjcha3112', db = 'test',charset = 'utf8')
curs = conn.cursor(pymysql.cursors.DictCursor)

job_seq = 0

while page < 100:
    page = page + 1 
    quest_urls = [] 
    try :
        driver.get(base_url + '&search.menuid=2&search.page='+ str(page) +'&userDisplay=50')
        driver.switch_to.frame('cafe_main') #iframe으로 프레임 전환 
        quest_list = driver.find_elements_by_css_selector('div.inner_list > a.article') 
        quest_urls = [ i.get_attribute('href')for i in quest_list ]
        print(len(quest_urls)) 
        for quest in quest_urls :
            try : 
                driver.get(quest) 
                driver.switch_to.frame('cafe_main')
                soup = bs(driver.page_source, 'html.parser') 
                #제목 추출
                title = soup.select('div.tit-box span.b')[0].get_text()
                print(title) 
                #내용 추출 
                content_tags = soup.select('#tbody')[0].select('p')
                content = ' '.join([ tags.get_text() for tags in content_tags ]) 
                print(content) 
                job_seq = job_seq+1 
                sqlInsert = "INSERT INTO schema_name.table_name VALUES (%s,%s,%s)" 
                val = (job_seq,title,content) 
                curs.execute(sqlInsert,val) 
                conn.commit()
                #말머리 추출
                try : 
                    tag = soup.select('div.tit-box span.head')[0].get_text() 
                    temp_list = [title, content]
                    f = open('preg_quest.csv', 'a+', encoding = 'ansi', newline='') 
                    wr = csv.writer(f) 
                    wr.writerow(temp_list)
                    f.close() 
                    cnt = cnt + 1
                except:
                    pass
            except: 
                        driver.switch_to_alert.accpet() 
                        driver.switch_to_alert
                        driver.switch_to_alert.accpet() 
    except : 
        pass
    print([page, cnt])
    time.sleep(2)
conn.close()

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/kyung/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


 
[1, 0]
[2, 0]
[3, 0]
[4, 0]


KeyboardInterrupt: 